# Najkrajše poti

Klasičen problem na grafih je iskanje najkrajših poti. Zanima nas na primer najkrajša pot med parom vozlišč $A$ in $B$ (*single-pair shortest path*). Naj bo ta najkrajša pot sestavljena iz vozlišč $A, ... X, B$, kjer je $X$ predzadnje vozlišče na poti. V tem primeru mora biti tudi pot od $A$ do $X$ najkrajša, sicer bi lahko pot od $A$ do $B$ izboljšali. Pri iskanju najkrajše poti od $A$ do $B$ posledično izračunamo tudi najkrajše poti do ostalih vozlišč na tej poti.

Če bomo že morali izračunati najkrajše poti iz $A$ do več drugih vozlišč, pa jih lahko izračunamo iz začetnega vozlišča kar do vseh (*single-source shortest path*). Opazimo tudi, da bodo te najkrajše poti v grafu formirale **drevo najkrajših poti**. Vsako vozlišče bo imelo namreč enega optimalnega predhodnika/starša na najkrajši poti (npr. $X$ bo predhodnik $B$-ja). Koren drevesa pa bo seveda v vozlišču $A$.

Za problem iskanja najkrajših poti med vsemi pari točk, lahko $N$-krat poženemo algoritem za iskanje drevesa najkrajših poti iz posameznega začetnega vozlišča. Obstajajo pa tudi drugi algoritmi, ki si namenjeni prav iskanju poti med vsemi pari točk. Tak primer je *Floyd-Warshall*-ov algoritem, ki ga bomo obravnavali kasneje.

Ukvarjali se bomo predvsem z neusmerjenimi grafi. V usmerjenih grafih je situacija namreč podobna in lahko uporabimo enake razmisleke.

Za začetek si pripravimo nekaj funkcij za izpis seznamov in za branje uteženih grafov. Funkcija read_graph prebere opis grafa (usmerjenega ali neusmerjenega) iz tekstovne datoteke in pripravi seznam povezav, seznam sosedov in matriko sosednosti.

In [1]:
#include <iostream>
#include <fstream>
#include <vector>
#include <queue>
#include <algorithm>
#include <array>
#include <tuple>
#include <iomanip>
using namespace std;

typedef pair<int,int> PII;
typedef array<int,3> III;
typedef vector<int> VI;
typedef vector<pair<int,int>> VII;
typedef vector<array<int,3>> VIII;
typedef vector<vector<int>> VVI;

const int INF=1'000'000'000;

In [2]:
void print(const vector<int> &sez, int w=4) {
    for (int x : sez) {
        cout << setw(w);
        if (x==INF) cout << "INF";
        else if (x==-INF) cout << "-INF";
        else cout << x;
        cout << " ";
    }
    cout << endl;
}

In [3]:
void print2(const vector<VI> &sez) {
    for (vector<int> row : sez) print(row);
}

In [4]:
auto read_graph(string fname, bool directed=true) {
    ifstream fin(fname);
    int n,m;
    fin >> n >> m;
    vector<III> edges;
    vector<VII> adj(n);
    vector<VI> mat(n,VI(n,INF));
    for (int i=0;i<m;i++) {
        int a,b,w;
        fin >> a >> b >> w;
        edges.push_back({a,b,w});
        adj[a].push_back({b,w});
        mat[a][b]=w;
        if (!directed) {
            edges.push_back({b,a,w});
            adj[b].push_back({a,w});
            mat[b][a]=w;
        }
    }
    return make_tuple(n, m, edges, adj, mat);
}

## Neuteženi grafi

V neuteženih grafih ni potrebe po kompliciranju, saj že poznamo metodo **iskanja v širino (BFS)**, ki obiskuje vozlišča od bližnjih proti bolj oddaljenim glede na število povezav. Potrebuje je malenkostno dopolnitev, da bo poleg obiskovanja vozlišč beležila še dolžine poti in prednike vozlišč v drevesu najkrajših poti.

<img alt="neutežen graf" src="graph.svg" width="250px">

In [5]:
ifstream fin("graph.txt");
int n,m;
fin >> n >> m;
vector<vector<int>> sosedi(n);
for (int i=0;i<m;i++) {
    int a,b;
    fin >> a >> b;
    sosedi[a].push_back(b);
    sosedi[b].push_back(a);
}

In [6]:
void BFS_distance(vector<VI> &adj, int start, vector<int> &dist, vector<int> &prev) {
    int n=adj.size();
    dist=vector<int>(n,-1); prev=vector<int>(n);
    vector<int> vis(n);
    queue<int> q;
    q.push(start); vis[start]=1;
    dist[start]=0; prev[start]=-1;
    while (!q.empty()) {
        int x=q.front(); q.pop();
        for (int y : adj[x]) {
            if (!vis[y]) {
                q.push(y); vis[y]=1;
                dist[y]=dist[x]+1; prev[y]=x;  // distance, previous node
            }
        }
    }
}

In [7]:
vector<int> dist, prev;
BFS_distance(sosedi,0,dist,prev);
print(dist); print(prev);

   0    1    3    2    1    2    3    2 
  -1    0    3    1    0    1    7    1 


## Uteženi grafi

V uteženih grafih pa je situacija malo bolj zapletena. Omejili se bomo na grafe s **pozitivnimi (nenegativnimi) utežmi**, s kakršnimi imamo večinoma opravka v praksi, kasneje pa se bomo vrnili še k negativnim utežem. Utež (ceno, dolžino) povezave med vozliščema $X$ in $Y$ bomo označili z $w(X,Y)$.

<img alt="utežen graf" src="weighted.svg" width="200px">

In [8]:
auto g = read_graph("weighted.txt", false);
int n=get<0>(g), m=get<1>(g);
vector<VII> adjw=get<3>(g);

### Dijkstrov algoritem

Tako kot smo v neuteženem primeru z iskanjem v širino računali najkrajše poti od bližnjih proti bolj oddaljenim vozliščem, bomo to storili tudi tu. Najbližje vozlišče je kar izhodiščno, $d(A) = 0$. Naslednje najbližje vozlišče pa bo eno od njegovih sosedov. Ker povezave niso negativne, je nemogoče, da bi dosegli manjšo razdaljo po kakšni poti z več povezavami. Tem neizračunanim sosedom do sedaj izračunanih vozlišč bomo rekli okolica. To so vozlišča, ki še niso izračunana in so iz že izračunanih dosegljiva po eni povezavi. Za vsako od njih bomo hranili potencialno najkrajšo pot $p(Y)$: kakšna bi bila razdalja, če bi se do njega premaknili z enega izmed že izračunanih vozlišč. Če iz okolice izberemo vozlišče $X$ s trenutno najmanjšo potencialno dolžino $p(X)$, bo to zagotovo dejanska najmanjša dolžina poti do tega vozlišča ($d(X) = p(X)$). Zaradi odsotnosti negativnih povezav, bi bila katerakoli druga pot od že izračunanih vozlišč do $X$ sestavljena iz več povezav in zato daljša. Množico že izračunanih vozlišč smo torej povečali z novim vozliščem $X$. Poskrbeti moramo še za posodobitev okolice. Vse sosede $Y$ vozlišča $X$ dodamo v okolico, če so že v njej, pa zgolj posodobimo njihovo potencialno oddaljenost z $p(Y) = \min(p(Y), d(X)+c(X,Y))$. Postopek ponavljamo, dokler nimamo izračunanih najkrajših poti do vseh vozlišč.

V postopku imamo opravka s tremi skupinami vozlišč. V prvi skupini so tista, za katera imamo že izračunane najkrajše poti. V drugi skupini so vozlišča iz okolice, ki imajo samo potencialne dolžine. Tretja skupina pa so še povsem neobiskana vozlišča. Pri implementaciji bomo vse te informacije hranili v tabeli potencialnih razdalj. Razdalja -1 bo označevala še neobiskano vozlišče iz tretje skupine, -2 pa že izračunano iz prve.

In [9]:
void Dijkstra(vector<VII> &adjw, int start, vector<int> &dist, vector<int> &prev) {
    int n=adjw.size();
    dist=vector<int>(n,-1); prev=vector<int>(n,-1);
    vector<int> p(n,-1);  // provisional distance (-1=unvisited, -2=done)
    p[start]=0;
    while (1) {
        int x=-1;  // smallest provisional
        for (int i=0;i<n;i++) if (p[i]>=0) {
            if (x==-1 || p[i]<p[x]) x=i;
        }
        if (x==-1) break;
        dist[x]=p[x]; p[x]=-2;
        for (auto [y,w] : adjw[x]) {  // update neighbors
            int d=dist[x]+w;
            if (p[y]==-1 || (p[y]>=0 && d<p[y])) { 
                p[y]=d; prev[y]=x; 
            }
        }
    }
}

In [10]:
vector<int> dist, prev;
Dijkstra(adjw,0,dist,prev);
print(dist); print(prev);

   0    4   11   17    9   22    7    8   11 
  -1    0    4    2    7    3    0    6    7 


Prostorska zahtevnost algoritma je $O(n)$. Časovna zahtevnost pa je odvisna od iskanja najmanjše potencialne razdalje ($O(n^2)$) in posodabljanja sosedov ($O(e)$). Ker je $e=O(n^2)$, je časovna zahtevnost take implementacije algoritma $O(n^2)$.

Razmislimo o izboljšavi. Težavno je iskanje vozlišča z najmanjšo potencialno razdaljo. Hkrati pa moramo biti sposobni posodabljati potencialne razdalje sosedov. Vozlišča iz okolice s potencialnimi razdaljami bi lahko hranili v uravnoteženem iskalnem drevesu. Tako lahko v času $O(\log n)$ poiščemo najmanjšega in spremenimo potencialno razdaljo vozlišča. Časovna zahtevnost bi bila $O(n \log n + e \log n) = O(e \log n)$.

Iskanje najmanjšega elementa je namen prioritetne vrste, zato je to v praksi pogostejši način implementacije, ki je tudi preprostejši in zato bolj učinkovit. Če za prioritetno vrsto uporabimo dvojiško kopico, mora ta omogočati tudi spremembo prioritete. Pravzaprav gre samo za zmanjšanje prioritete v minimalni dvojiški kopici, kar lahko dosežemo v logaritemskem času. Tudi ta rešitev ima časovno zahtevnost $O(e \log n)$.

V spodnji implementaciji pa bomo malo "goljufali" in se izognili spreminjanju prioritet. Pri posodabljanju bomo v prioritetno vrsto samo vstavili novo manjšo vrednost, stare pa ne bomo izbrisali. Nova vrednost bo prišla iz vrste prej, zato lahko stare neveljavne vrednosti, ki pridejo iz vrste nekoč kasneje, enostavno ignoriramo. V tabeli razdalj `dist` bomo hranili razdalje do vseh vozlišč (nekatere so pravilne, druge zgolj potencialne). Vozlišča, katerih razdalje so zgolj potencialne, bomo hranili v prioritetni vrsti. Ko pride vozlišče iz prioritetne vrste, vemo, da je njegova razdalja pravilna in posodobimo sosede. V prioritetni vrsti je lahko $O(e)$ elementov, zato je taka tudi prostorska zahtevnost. Časovna zahtevnost pa je $O(e \log e) = O(e \log n^2) = O(e \cdot 2\log n) = O(e \log n)$. Goljufija torej ni bila prav huda.

Vso to kompliciranje pa ima smisel samo, če je graf dovolj redek. Če je graf gost in vsebuje skoraj vse možne povezave ($e \approx n^2$), je časovna zahtevnost $O(e \log n)$ pravzaprav $O(n^2 \log n)$, kar je slabše od $O(n^2)$, s čimer smo začeli.

In [11]:
void Dijkstra_PQ(vector<VII> &adjw, int start, vector<int> &dist, vector<int> &prev) {
    int n=adjw.size();
    dist=vector<int>(n,-1); prev=vector<int>(n,-1);
    priority_queue<PII, vector<PII>, greater<PII>> pq;  // (distance, node)
    dist[start]=0; pq.push({0,start});
    while (!pq.empty()) {
        auto [d,x]=pq.top(); pq.pop();
        if (dist[x]!=d) continue;  // ignore old values
        for (auto [y,w] : adjw[x]) {  // update neighbors
            int d=dist[x]+w;
            if (dist[y]==-1 || d<dist[y]) {
                dist[y]=d; prev[y]=x;
                pq.push({d,y});
            }
        }
    }
}

In [12]:
vector<int> dist, prev;
Dijkstra_PQ(adjw,0,dist,prev);
print(dist); print(prev);

   0    4   11   17    9   22    7    8   11 
  -1    0    4    2    7    3    0    6    7 


Algoritem lahko v nekaterih primerih še izboljšamo. Pogosto so uteži relativno majhna cela števila. Naj bo $c$ največja utež v grafu. Največja oddaljenost vozlišča v grafu bo tako $(n-1)c$. Namesto v prioritetni vrsti lahko vozlišča s potencialnimi razdaljami hranimo "popredalčkana" v tabeli, ki na mestu $i$ hrani seznam vozlišč na razdalji $i$. Temu rečemo tudi vrsta z vedri (*bucket queue*). Podobno kot prej ne spreminjamo vrednosti, ampak dodajamo nove in po potrebi ignoriramo stare. Prostorska in časovna zahtevnost take rešitve sta $O(e + nc)$. Reče se mu tudi Dialov algoritem.

In [13]:
void Dijkstra_BQ(vector<VII> &adjw, int start, vector<int> &dist, vector<int> &prev) {
    int n=adjw.size();
    dist=vector<int>(n,-1); prev=vector<int>(n,-1);
    int c=0;  // maximum weight
    for (int x=0;x<n;x++) for (auto [y,w] : adjw[x]) c=max(c, w);
    int maxd=(n-1)*c;
    vector<VI> bq(maxd+1);  // bucket queue
    dist[start]=0; bq[0].push_back(start);
    for (int d=0;d<=maxd;d++) {
        for (int x : bq[d]) {
            if (dist[x]!=d) continue;  // ignore old values
            for (auto [y,w] : adjw[x]) {  // update neighbors
                int d=dist[x]+w;
                if (dist[y]==-1 || d<dist[y]) {
                    dist[y]=d; prev[y]=x;
                    bq[d].push_back(y);
                }
            }
        }
    }
}

In [14]:
vector<int> dist, prev;
Dijkstra_BQ(adjw,0,dist,prev);
print(dist); print(prev);

   0    4   11   17    9   22    7    8   11 
  -1    0    4    2    7    3    0    6    7 


#### Negativne uteži

Do sedaj smo se omejili na pozitivne oz. nenegativne uteži. Negativne uteži imajo smisel samo na usmerjenih grafih. Sicer bi se lahko sprehajali tja in nazaj po isti negativni povezavi in imeli vedno krajšo pot.

Kje pa pride do težave na usmerjenih grafih? Naša predpostavka, da ima vozlišče v okolici z najmanjšo potencialno razdaljo prav tako tudi dejansko razdaljo, ni več resnična. To lahko demonstriramo s spodnjim primerom.

In [15]:
// (0,1,2), (0,2,3), (2,1,-2)
vector<VII> adjw = {{{1,2},{2,3}},{},{{1,-2}}};
vector<int> dist, prev;
Dijkstra(adjw,0,dist,prev);
print(dist); print(prev);

   0    2    3 
  -1    0    0 


Situacija je lahko še slabša. V usmerjenem grafu se lahko pojavi negativen cikel (cikel z negativno vsoto uteži). V takem primeru koncept najkrajših poti tudi nima smisla, ker lahko krožimo po ciklu in s tem poljubno krajšamo svojo pot. 

Obstajajo algoritmi, ki uspešno rešujejo probleme najkrajših poti tudi v prisotnosti negativnih povezav in zaznavajo prisotnost negativnih ciklov. Klasičen primer je Bellman-Fordov algoritem, ki ga bomo obravnavali kasneje.

## Primeri

Grafi so zelo pogost način modeliranja relacij, iskanje najkrajših poti pa eden najobičajnejših problemov na njih. V nadaljevanju si bomo ogledali nekaj primerov sorodnih problemov.

### Najširša pot

Recimo, da z grafom modeliramo cestno omrežje. Povezave predstavljajo dvosmerne ceste, vozlišča pa križišča. Uteži povezav ustrezajo širini ceste. Kakšna je največja širina vozila, ki se lahko pripelje od vozlišča $A$ do $B$?

Gre za problem iskanja najširše poti (*widest path, maximum capacity path*). Pri njem iščemo pot od $A$ do $B$, za katero bo veljalo, da je najmanjša utež na poti čim večja. Za primerjavo nas je v klasičnem problemu najkrajših poti zanimala tista pot, kjer je bila vsota uteži čim manjša. Vsoto smo torej zamenjali z minimumom, minimizacijo pa z maksimizacijo.

Uporabimo lahko povsem enak razmislek kot pri Dijkstrovem algoritmu. Poti do vozlišče bomo računali od širših proti ožjim. Najširša pot ($\infty$ širine) vodi do začetnega vozlišča. Na vsakem koraku bomo med izračunana vozlišča dodali vozlišče iz okolice, do katerega vodi trenutno najširša potencialna pot. To ima zagotovo pravo vrednost, saj bi kakršnakoli druga pot obiskala več povezav, kar širine poti ne more povečati, temveč jo kvečjemu zmanjša.

### Najdaljša pot

Kaj pa, če nas namesto najkrajše poti zanima najdaljša? Trivialno, uteži negiramo in je problem rešen. Žal ne, ker s tem dobimo graf z negativnimi cikli. Pravzaprav je koncept najdaljše poti slabo definiran - lahko bi se sprehajali sem in tja po isti povezavi in poljubno podaljšali pot.

V primeru najdaljše poti nas zanimajo poti brez ponovljenih vozlišč. Pri najkrajših poteh je bilo to samoumevno, saj od večkratnega obiskovanja vozlišč ni nobene koristi ampak samo škoda. Izkaže se, da gre za težek problem, ki spada v razred NP-polnih (*NP-complete*) problemov. Več o tem pa pri predmetu Izračunljivost in računska zahtevnost.

Izjema so usmerjeni aciklični grafi (DAG), ki ne vsebujejo ciklov. Tam smo že rešili prav ta problem, le da smo mu rekli kritična pot.

### 15 Puzzle

Verjetno poznate drsno sestavljanko prikazano na spodnji sliki. Igra se na mreži velikosti 4x4, kjer se na vsakem polju nahaja ploščica z enim izmed števil od 1 do 15. Vsako število se pojavi enkrat, eno polje pa je prazno. Zanima nas, kako naj s premiki ploščic na prazno sosednje polje uredimo števila po velikosti (po vrsticah od zgoraj navzdol in znotraj vrstice od leve proti desni). Še bolje, izračunajmo najmanjše potrebno število potez.

<img alt="15 puzzle (wikipedija)" src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/15-puzzle_magical.svg/600px-15-puzzle_magical.svg.png" width="150px">

V tem primeru nimamo opravka z **grafom stanj**. Vsako stanje sestavljanke ustreza nekemu vozlišču. Za izračun najmanjšega števila potez bomo uporabili iskanje v širino (BFS). Seveda ne bomo vnaprej zgradili celotnega grafa, ker bi bil ta prevelik, ampak ga bomo odkrivali sproti. Rečemo, da bo graf predstavljen *implicitno* s stanji sestavljanke. Za vsako stanje oz. vozlišče znamo namreč izračunati njegove sosede. Pri tem upamo, da bomo dosegli rešitev dovolj zgodaj, preden bomo preiskali prevelik del grafa.

Obstajajo tudi izboljšave tega osnovnega preiskovanja, ki z uporabo hevristik usmerjajo iskanje proti delom grafa, v katerih je bolj verjetno, da bomo našli rešitev. Primer nadgradnje iskanja najkrajših poti z uporabi hevristik je algoritem *A\**.

In [16]:
int puzzle15(VVI start, vector<VVI> &seq) {
    map<VVI, int> dist;
    map<VVI, VVI> prev;
    queue<VVI> q;
    q.push(start); dist[start]=0;
    VVI goal = {{1,2,3,4},{5,6,7,8},{9,10,11,12},{13,14,15,0}};
    while (!q.empty()) {
        VVI state=q.front(); q.pop();
        if (state == goal) break;
        // next states
        for (int i=0;i<4;i++) for (int j=0;j<4;j++) if (state[i][j]==0) {  // find empty cell
            for (auto [di,dj] : VII{{0,1},{0,-1},{1,0},{-1,0}}) {  // possible moves
                int i2=i+di, j2=j+dj;
                if (i2<0 || i2>=4 || j2<0 || j2>=4) continue;
                VVI state2=state;  // adjacent state
                swap(state2[i][j], state2[i2][j2]);
                if (dist.count(state2)==0) {  // new?
                    dist[state2] = dist[state]+1;
                    prev[state2] = state;
                    q.push(state2);
                }
            }
        }
    }
    // reconstruct sequence of states
    VVI state=goal;
    seq.push_back(state);
    while (state!=start) {
        state = prev[state];
        seq.push_back(state);
    }
    reverse(seq.begin(),seq.end());
    return dist[goal];
}

In [17]:
VVI state = {{5, 0, 2, 3},
             {6, 1, 7, 4},
             {9, 10,11,8},
             {13,14,15,12}};
vector<VVI> seq;
cout << puzzle15(state, seq) << endl;
for (VVI state : seq) {
    cout << endl;
    print2(state);
}

9

   5    0    2    3 
   6    1    7    4 
   9   10   11    8 
  13   14   15   12 

   5    1    2    3 
   6    0    7    4 
   9   10   11    8 
  13   14   15   12 

   5    1    2    3 
   0    6    7    4 
   9   10   11    8 
  13   14   15   12 

   0    1    2    3 
   5    6    7    4 
   9   10   11    8 
  13   14   15   12 

   1    0    2    3 
   5    6    7    4 
   9   10   11    8 
  13   14   15   12 

   1    2    0    3 
   5    6    7    4 
   9   10   11    8 
  13   14   15   12 

   1    2    3    0 
   5    6    7    4 
   9   10   11    8 
  13   14   15   12 

   1    2    3    4 
   5    6    7    0 
   9   10   11    8 
  13   14   15   12 

   1    2    3    4 
   5    6    7    8 
   9   10   11    0 
  13   14   15   12 

   1    2    3    4 
   5    6    7    8 
   9   10   11   12 
  13   14   15    0 


Za konec zgolj kot zanimivost omenimo še reševanje **Rubikove kocke**. Iskanje najkrajših poti je še vedno predmet algoritmičnega raziskovanja. S precej računske moči so nedavno dokazali, da je mogoče vsako stanje Rubikove kocke rešiti v največ [20 potezah](http://www.cube20.org/) oz. [26 potezah](http://www.cube20.org/qtm/) (če je ena poteza rotacija ploskve samo za 90° in ne 180°).

## Bellman-Fordov algoritem

Vrnimo se k težavi z negativnimi utežmi v usmerjenem grafu. Če ne obstaja negativen cikel, negativne uteži še ne predstavljajo razloga, da ne bi mogli rešiti problema najkrajše poti med dvema vozliščema. Tudi tokrat bomo izračunali najkrajše poti od izhodiščnega vozlišča do vseh ostalih. Samo reševanja se bomo morali lotiti kako drugače kot z Dijkstrovim algoritmom. Za začetek predpostavimo, da v grafu ni negativnega cikla (cikla z negativno vsoto uteži), kasneje pa bomo naslovili tudi to.

Poskuse bomo delali na podobnem grafu kot prej, le da bomo nekoliko spremenili uteži, da bodo usmerjene in lahko tudi negativne.

<img alt="graf z negativnimi utežmi" src="negative.svg" width="200px">

In [18]:
auto g = read_graph("negative.txt", true);
int n=get<0>(g), m=get<1>(g);
vector<III> edges=get<2>(g);
vector<VII> adjw=get<3>(g);
vector<VI> mat=get<4>(g);

Če graf ne vsebuje negativnega cikla, potem se na najkrajših poteh vozlišča ne ponavljajo, torej imajo najkrajše poti največ $n-1$ povezav. Najkrajše poti bomo računali po naraščajočem številu uporabljenih povezav. Naj bo $d_l(X)$ najkrajša pot od izhodiščnega vozlišča $A$ do vozlišča $X$, ki uporabi kvečjemu $l$ povezav. Če poznamo rezultate $d_{l-1}(*)$ za vsa vozlišča, lahko iz njih izračunamo $d_l(X)$. Najkrajša pot, ki uporabi kvečjemu $l$ povezav, lahko uporabi kvečjemu $l-1$ povezav, ali pa z zadnjo povezavo podaljša neko najkrajšo pot do $Y$ z največ $l-1$ povezavami še z vozliščem $X$. To pa pravzaprav pomeni, da lahko obravnavamo vse povezave $(Y,X)$ in z vsako povezavo poskusimo popraviti $d_{l}(X)$ s pomočjo rezultata $d_{l-1}(Y)$. Ker so tu smiselne tudi negativne razdalje, si bomo nedosegljiva vozlišča označili z $d_l(X)=\infty$.

$$
d_l(X) = \min
\begin{cases}
d_{l-1}(Y) \\
\min_Y (d_{l-1}(Y) + w(Y,X))
\end{cases}
$$

Ker vemo, da ne bomo nikoli potrebovali več kot $n-1$ povezav, se bo rešitev nahajala v $d_{n-1}(X)$.

Pazljivi moramo biti na vsote $\infty + w$, če je $w < 0$. V teoriji je rezultat še vedno $\infty$. Če pa v praksi za neskončno uporabljamo neko veliko konstanto, bomo dobili za rezultat samo malenkost manjšo konstanto. Enostavno preskočimo obravnavo povezav, ki vodijo iz vozlišč na trenutno neskončni razdalji.

Vrednosti $d_l$ bomo iz $d_{l-1}$ računali v nekoliko drugačnem vrstnem redu. Zgornja formula prikazuje, kako bi za izbrano vozlišče $X$ izračunali $d_l(X)$ z obravnavo vseh sosednjih vozlišč $Y$, iz katerih kaže povezava v $X$. Tako bi lahko izračunali najprej $d_l(X_1)$, nato $d_l(X_2)$, itd. Namesto tega bomo računali vse $d_l$ hkrati z zaporedno obravnavo povezav v grafu v nekem poljubnem vrstnem redu. Ko imamo opravka s povezavo $Y \rightarrow X$, poskusimo izboljšati rezultat $d_l(X)$ glede na $d_{l-1}(Y) + w(Y,X)$.

In [19]:
void BellmanFord(int n, vector<III> &edges, int start, vector<int> &dist) {
    dist=vector<int>(n,INF);
    vector<vector<int>> d(n,vector<int>(n,INF));
    d[0][start]=0;
    for (int l=1;l<=n-1;l++) {
        for (int x=0;x<n;x++) d[l][x]=d[l-1][x];
        for (auto [y,x,w] : edges) {
            if (d[l-1][y]==INF) continue;
            int dx=d[l-1][y]+w;
            if (dx<d[l][x]) { d[l][x]=dx; }
        }
        cout << "l=" << l << ":";
        print(d[l]);
    }
    dist=d[n-1];
}

In [20]:
vector<int> dist;
BellmanFord(n,edges,0,dist);
print(dist);

l=1:   0    4  INF  INF  INF  INF   -7  INF  INF 
l=2:   0    4    5  INF  INF  INF   -7    3  INF 
l=3:   0    4    5   11    7  INF   -7    3    0 
l=4:   0    4    5   11    7   12   -7    3    0 
l=5:   0    4    5   -8    7   12   -7    3    0 
l=6:   0    4    5   -8    2   12   -7    3    0 
l=7:   0    4    5   -8    2   12   -7    3    0 
l=8:   0    4    5   -8    2   12   -7    3    0 
   0    4    5   -8    2   12   -7    3    0 


Zavoljo ilustracije dogajanja smo sproti izpisovali izračunane razdalje. Tokrat vidimo, da imamo tudi negativne razdalje do nekaterih vozlišč. Zanimivo je npr. vozlišče 3, ki je na razdalji -8 in do katerega vodi pot 0, 6, 7, 8, 5, 3.

Časovna zahtevnost algoritma je očitno $O(en)$, ker naredimo $n-1$ iteracij in v vsaki obravnavamo $e$ povezav.

**Prostorska zahtevnost** v zgornjem primeru je $O(n^2)$, kar pa lahko izboljšamo. Namesto, da hranimo celotno tabelo $d$, bi lahko hranili samo zadnjo vrstico tabele $d_{l-1}$, iz nje izračunali $d_l$ ter prejšnje rešitve pozabili oz. reciklirali prostor za kasnejše izračune. Taka rešitev bi zahtevala samo $O(n)$ prostora. Običajno pa se na prostoru prihrani še malo s tem, da se ne računa ločenih vrednosti $d_l$, ampak se vse vpisuje kar v isto tabelo $d$. Tako izračunane vrednosti nimajo več direktne interpretacije kot najkrajše poti z največ $l$ povezavami. Ker lahko isto vrednost popravimo večkrat v isti iteraciji, bodo verjetno hitreje konvergirale h končnemu rezultatu, še vedno pa lahko v najslabšem primeru potrebujemo $n-1$ iteracij. 

Enako kot pri Dijkstrovem algoritmu, si lahko tudi tu beležimo predhodnike vozlišča na najkrajši poti za rekonstrukcijo najkrajše poti.

In [21]:
void BellmanFord(int n, vector<III> &edges, int start, vector<int> &dist, vector<int> &prev) {
    dist=vector<int>(n,INF); prev=vector<int>(n,-1);
    dist[start]=0;
    for (int l=1;l<=n-1;l++) {
        for (auto [y,x,w] : edges) {
            if (dist[y]==INF) continue;
            int dx=dist[y]+w;
            if (dx<dist[x]) { dist[x]=dx; prev[x]=y; }
        }
    }
}

In [22]:
vector<int> dist, prev;
BellmanFord(n,edges,0,dist,prev);
print(dist); print(prev);

   0    4    5   -8    2   12   -7    3    0 
  -1    0    1    5    3    8    0    6    7 


Algoritem je sedaj že elegantno kratek in prostorsko učinkovit. Na voljo pa je še nekaj možnosti za **optimizacijo časovne zahtevnosti**. Če v neki iteraciji algoritma pri obravnavi vseh povezav ne pride do nobene spremembe, tudi v kasnejših iteracijah ne bo, zato ga lahko predčasno prekinemo. V najslabšem primeru bomo še vedno potrebovali $n-1$ iteracij, v praksi pa precej manj. Če graf vsebuje negativen cikel, se tak algoritem ne bo ustavil (seveda pa ga lahko prekinemo po $n$ iteracijah in javimo negativen cikel).

In [23]:
void BellmanFord_NoNegCycle(int n, vector<III> &edges, int start, vector<int> &dist, vector<int> &prev) {
    dist=vector<int>(n,INF); prev=vector<int>(n,-1);
    dist[start]=0;
    bool change=true;
    while (change) {
        change=false;
        for (auto [y,x,w] : edges) {
            if (dist[y]==INF) continue;
            int dx=dist[y]+w;
            if (dx<dist[x]) { dist[x]=dx; prev[x]=y; change=true; }
        }
    }
}

In [24]:
vector<int> dist, prev;
BellmanFord_NoNegCycle(n,edges,0,dist,prev);
print(dist); print(prev);

   0    4    5   -8    2   12   -7    3    0 
  -1    0    1    5    3    8    0    6    7 


Razmislimo še o povezavah, s katerimi se moramo ukvarjati v posamezni iteraciji. Če v predhodni iteraciji nismo spremenili najkrajše poti do vozlišča $Y$, potem v naslednji iteraciji nima smisla obravnavati izhodnih povezav iz tega vozlišča, ker ne bo prišlo do nobenega popravka. Prav nam bo prišel seznam sosedov, ker ne bomo več obravnavali vseh povezav v poljubnem vrstnem redu, ampak samo povezave iz relevantnih vozlišč. Spremenjena vozlišča bomo hranili kar v vrsti in pazili, da ne dodamo istega vozlišča večkrat, če se ta že nahaja v vrsti. Izboljšava je znana pod imenom Bellman-Ford-Moore. Časovna zahtevnost v najslabšem primeru je še vedno $O(en)$, v praksi pa je algoritem bistveno hitrejši.

In [25]:
void BellmanFordMoore(int n, vector<VII> &adjw, int start, vector<int> &dist, vector<int> &prev) {
    dist=vector<int>(n,INF); prev=vector<int>(n,-1);
    dist[start]=0;
    queue<int> changed; changed.push({start});
    vector<bool> in_queue(n); in_queue[start]=1;
    while (!changed.empty()) {
        int y=changed.front(); changed.pop();
        in_queue[y]=false;
        for (auto [x,w] : adjw[y]) {
            int dx=dist[y]+w;
            if (dx<dist[x]) {
                dist[x]=dx; prev[x]=y;
                if (!in_queue[x]) { changed.push(x); in_queue[x]=true; }
            }
        }
    }
}

In [26]:
vector<int> dist, prev;
BellmanFordMoore(n,adjw,0,dist,prev);
print(dist); print(prev);

   0    4    5   -8    2   12   -7    3    0 
  -1    0    1    5    3    8    0    6    7 


### Negativni cikli

Do sedaj smo predpostavljali, da graf ne vsebuje negativnega cikla. Kaj pa se zgodi v njegovi prisotnosti? Ga lahko kako zaznamo?

Če imamo opravka z negativnim ciklom, se bodo neke razdalje krajšale v neskončnost. Brez negativnega cikla se mora spreminjanje razdalj zagotovo zaključiti po $n-1$ iteracijah, ker poti daljše od $n-1$ povezav gotovo vsebujejo cikel. Če izvedemo še $n$-to iteracijo algoritma Bellman-Ford in ugotovimo, da je prišlo do kakšne spremembe, potem graf vsebuje negativen cikel, ki je dosegljiv iz začetnega vozlišča $A$. Če negativen cikel ni dosegljiv iz začetnega vozlišča, ga ne bomo zaznali, hkrati pa tudi ne bo povzročal težav pri računanju najkrajših poti.

V prejšnjem odstavku smo samo zaznali negativen cikel, sedaj pa ga še poiščimo. Naj bo $X$ poljubno vozlišče, pri katerem je prišlo do spremembe v $n$-ti iteraciji. To vozlišče je lahko del negativnega cikla ali pa je to neko vozlišče, ki je dosegljivo iz negativnega cikla. Če se iz vozlišča $X$ premikamo po predhodnikih, bomo gotovo prišli na cikel in se začeli krožiti po njem. Premikanje na predhodnika lahko ponovimo, dokler ne obiščemo nekega vozlišča $Y$ dvakrat. To vozlišče je sedaj gotovo del cikla, našli pa ga bomo v največ $n$ korakih. Lahko naredimo še en krog, da sestavimo cikel, ali pa iz seznama že obiskanih vozlišč odstranimo tista, ki smo jih obiskali, da smo prišli prvič od $X$ do $Y$.

In [27]:
void BellmanFord_NegCycle(int n, vector<III> &edges, int start, vector<int> &dist, vector<int> &prev) {
    dist=vector<int>(n,INF); prev=vector<int>(n,-1);
    dist[start]=0;
    for (int l=1;l<=n-1;l++) {
        for (auto [y,x,w] : edges) {
            if (dist[y]==INF) continue;
            int dx=dist[y]+w;
            if (dx<dist[x]) { dist[x]=dx; prev[x]=y; }
        }
    }
    // extra loop to detect a negative cycle
    int a=-1;
    for (auto [y,x,w] : edges) {
        if (dist[y]==INF) continue;
        int dx=dist[y]+w;
        if (dx<dist[x]) { dist[x]=dx; prev[x]=y; a=x; }
    }
    if (a!=-1) {  // cycle detected
        cout << "Unexpected distance change at node: " << a << endl;
        vector<int> vis(n,-1), seq;
        int i=0;
        while (vis[a]==-1) {  // find cycle
            vis[a]=i; seq.push_back(a);
            a=prev[a];
            i++;
        }
        cout << "Node on a cycle: " << a << endl;
        vector<int> cyc(seq.begin()+vis[a], seq.end());
        reverse(cyc.begin(), cyc.end());
        cout << "Negative cycle:";
        for (int x : cyc) cout << " " << x;
        cout << endl;
    }
}

Algoritem lahko preizkusimo na skoraj enakem grafu kot prej. Spremenili bomo samo utež povezave iz vozlišča 7 do vozlišča 1 in sicer iz 6 na -6. Tako nastane negativen cikel z vozlišči 1, 2, 4, 7, ki ima vsoto povezav -2.

Vidimo lahko, da algoritem zazna spremembo razdalje na vozlišču 3 (in še nekaterih drugih). Od tam se premika k predhodnikom na najkrajši poti dokler na ciklu drugič ne obišče vozlišča 7. Zadnji del poti med obema obiskoma vozlišča 7 predstavlja cikel, le da v obratno smer.

In [28]:
for (auto& [x,y,w] : edges) {
    if (x==7 && y==1) w*=-1;
}

In [29]:
vector<int> dist,prev;
BellmanFord_NegCycle(n,edges,0,dist,prev);
print(dist); print(prev);

Unexpected distance change at node: 3
Node on a cycle: 7
Negative cycle: 1 2 4 7
   0  -11   -8  -16   -6    4   -7   -5   -8 
  -1    7    1    5    2    8    0    4    7 


## Floyd-Warshallov algoritem

Poiskati znamo najkrajše poti iz enega do vseh ostalih vozlišč. Kaj pa **najkrajše poti med vsemi pari vozlišč** (*all-pair shortest paths*)? Seveda lahko za vsako izhodiščno vozlišče uporabimo katerega od že znanih algoritmov. Če primerjamo časovne zahtevnost, ki jih na tak način dobimo z uporabo prave različice algoritma za goste in redke grafe, bomo ugotovili, da je Dijkstrov algoritem boljša izbira. Je pa zato Bellman-Fordov algoritem sposoben pravilno obravnavati negativne povezave. Temu naboru bomo dodali še Floyd-Warshallov algoritem, ki je namenjen iskanju najkrajših poti med vsemi pari vozlišč v grafih, ki lahko vsebujejo tudi negativne povezave. V primerjavi z Bellman-Fordom pa je bolj učinkovit na gostih grafih.

| algoritem       | gosti grafi [$e=O(n^2)$] | redki grafi [$e=O(n)$]  |
| --------        | -------                  | -------                 |
| Dijkstra        | $O(n \cdot n^2)=O(n^3)$  | $O(n \cdot e\log{n})=O(n^2 \log{n})$       |
| Bellman-Ford    | $O(n \cdot en) = O(n^4)$ | $O(n \cdot en) = O(n^3)$ |
| Floyd-Warshall  | $O(n^3)$                 | $O(n^3)$                |


Računali bomo najkrajše poti med vozlišči $x$ in $y$, kar bomo označili z $f(x,y)$. Za začetek predpostavimo, da graf ne vsebuje negativnih povezav. Tokrat bomo najkrajše poti obravnavali po naraščajočem naboru vmesnih vozlišč, ki jih lahko najkrajša pot uporabi: $f_k(x,y)$ naj predstavlja najkrajšo pot od vozlišča $x$ do $y$, ki lahko vmes uporabi samo vozlišča iz množice ${1,2,\ldots,k}$ (prvih $k$ vozlišč po nekem poljubnem vnaprej definiranem vrstnem redu). Najkrajša pot med vozliščema $x$ in $y$ po prvih $k$ vmesnih vozliščih lahko sploh ne uporabi $k$-tega vozlišča, ker je dovolj dobra že pot po prvih $k-1$ vmesnih vozliščih. Če ga uporabi, pa mora priti iz $x$ do $k$ in nato od $k$ do $y$ po prvih $k-1$ vozliščih (zagotovo ni smiselno vozlišča $k$ obiskati več kot enkrat, če ni negativnih ciklov). Z dolžinami povezav lahko inicializiramo vrednosti $f_0(x,y)$, ki ne uporabijo nobenega vmesnega vozlišča, razdalje $f_0(x,x)$ pa nastavimo na $0$. Po izvedbi algoritma se bodo dolžine najkrajših poti nahajale v vrednostih $f_n(x,y)$.

$$
f_k(x,y) = \min
\begin{cases}
f_{k-1}(x,y) \\
f_{k-1}(x,k) + f_{k-1}(k,y)
\end{cases}
$$

Izgleda, kot da bomo potrebovali $O(n^3)$ prostora, vendar lahko tudi tu prihranimo na prostoru, če vrednosti kar prepisujemo v 2D tabeli $f(x,y)$. Ko računamo rešitev $f(x,y)$ s $k$ vmesnimi vozlišči, imamo na njenem mestu že rešitev s $k-1$ vozlišči. Sicer pa jo poskusimo izboljšati s potjo po $k-1$ vmesnih vozliščih in vmesnim postankom v vozlišču $k$. Za to bi potrebovali vrednosti $f_{k-1}(x,k)$ in $f_{k-1}(k,y)$, ki pa ju nimamo shranjenih ločeno — na mestu $f(x,k)$ se nahaja vrednost $f_{k-1}(x,k)$ ali pa že popravljena vrednost $f_{k}(x,k)$ iz trenutne iteracije. Na srečo sta obe možnosti enaki, ker za pot od $x$ do $k$ nič ne pomaga vmesni postanek v $k$.

Časovna zahtevnost algoritma je $O(n^3)$, prostorska pa $O(n^2)$.

In [30]:
void FloydWarshall(vector<VI> &mat, vector<VI> &dist) {
    int n=mat.size();
    dist=mat;
    for (int x=0;x<n;x++) dist[x][x]=0;
    for (int k=0;k<n;k++) {
        for (int x=0;x<n;x++) {
            for (int y=0;y<n;y++) {
                dist[x][y]=min(dist[x][y], dist[x][k]+dist[k][y]);
            }
        }
    }
}

Algoritem bomo preizkusili na neusmerjenem grafu s pozitivnimi utežmi. Tokrat nam pride najbolj prav predstavitev grafa z matriko sosednosti. Ker bo prostorska zahtevnost algoritma kvadratna, ni nobene škode, če ima tudi predstavitev grafa kvadratno prostorsko zahtevnost. Hitro lahko preverimo, da se rezultati v prvi vrstici ujemajo z najkrajšimi potmi, ki smo jih izračunali z Dijkstrovim algoritmom.

In [31]:
auto g = read_graph("weighted.txt", false);
int n=get<0>(g), m=get<1>(g);
vector<VI> mat=get<4>(g);

In [32]:
vector<VI> dist;
FloydWarshall(mat,dist);
print2(dist);

   0    4   11   17    9   22    7    8   11 
   4    0    9   15   11   20   11   12   15 
  11    9    0    6    2   11    4    3    6 
  17   15    6    0    8    5   10    9   12 
   9   11    2    8    0   13    2    1    4 
  22   20   11    5   13    0   15   14   12 
   7   11    4   10    2   15    0    1    4 
   8   12    3    9    1   14    1    0    3 
  11   15    6   12    4   12    4    3    0 


Kako lahko **rekonstruiramo** najkrajšo pot med nekim začetnim vozliščem $x$ in končnim vozliščem $y$? Po izvedbi algoritma imamo v $f(*,*)$ izračunane najkrajše poti med vozlišči. To lahko izkoristimo, da poiščemo naslednje vozlišče $z$ na najkrajši poti od $x$ do $y$. To bo sosed vozlišča $x$, za katerega velja $w(x,z) + f(z,y) = f(x,y)$. Tako lahko rekonstruiramo najkrajšo pot v $O(e)$, ker bomo vsako povezavo obravnavali največ enkrat.

Lahko pa smo bolj učinkoviti. Večino časa v prejšnji rekonstrukciji je namenjenega iskanju naslednjega vozlišča, ki vodi do izračunane razdalje. Podobno kot pri Bellman-Fordu, lahko tudi tu hranimo predhodnje/predzadnje vozlišče na vsaki izračunani najkrajši poti, ki nam bo prišlo prav pri rekonstrukciji. Če smo ravnokar obravnavali vmesno vozlišče $k$ in našli boljšo pot $f(x,y)=f(x,k)+f(k,y)$, bo predhodnik $p(x,y)$ kar enak predhodniku $p(k,y)$.

Algoritem nima težav z **negativnimi povezavami**.  Paziti pa moramo, da ne dobimo iz $\infty + (-1)$ rezultata $\infty - 1$. 

Kaj pa se zgodi v primeru **negativnega cikla**? Algoritem se normalno zaključi, vprašanje je samo, ali so izračunane vrednosti smiselne in kako lahko to ugotovimo. Če obstaja negativen cikel, ki vključuje vozlišče $x$, bomo med izvajanjem algoritma izračunali najkrajšo pot $f(x,x) < 0$. Tako najdemo vsa vozlišča, ki so del nekega negativnega cikla (ne nujno istega), poleg njih pa morda še kakšna (ne nujno vsa), ki so dosegljiva iz negativnega cikla. Prisotnost negativnega cikla lahko torej enostavno zaznamo. Te razdalje so lahko načeloma $-\infty$ in med računanjem tudi eksponentno hitro padajo. Zato moramo v praksi navzdol omejiti izračunane vrednosti na neko dovolj negativno konstanto.

In [33]:
void FloydWarshall(vector<VI> &mat, vector<VI> &dist, vector<VI> &prev) {
    int n=mat.size();
    dist=vector<VI>(n,vector<int>(n,INF));
    prev=vector<VI>(n,vector<int>(n,-1));
    for (int x=0;x<n;x++) {
        for (int y=0;y<n;y++) {
            dist[x][y]=mat[x][y];
            prev[x][y]=x;
        }
        dist[x][x]=0;
        prev[x][x]=-1;
    }
    for (int k=0;k<n;k++) {
        for (int x=0;x<n;x++) {
            for (int y=0;y<n;y++) {
                if (dist[x][k]==INF || dist[k][y]==INF) continue;
                int dxy=dist[x][k]+dist[k][y];
                dxy=max(dxy, -INF);
                if (dxy<dist[x][y]) {
                    dist[x][y]=dxy;
                    prev[x][y]=prev[k][y];
                }
                dist[x][y]=min(dist[x][y], dist[x][k]+dist[k][y]);
            }
        }
    }
    for (int x=0;x<n;x++) {
        if (dist[x][x]<0) {
            cout << "Negative cycle detected." << endl;
            break;
        }
    }
}

In [34]:
auto g = read_graph("negative.txt", true);
int n=get<0>(g), m=get<1>(g);
vector<VI> mat=get<4>(g);

In [35]:
vector<VI> dist, prev;
FloydWarshall(mat,dist,prev);
int x=0, y=3;
cout << "f(" << x << "," << y << ") = " << dist[x][y] << endl;
vector<int> path={y};
while (y!=x) {
    y=prev[x][y];
    path.push_back(y);
}
reverse(path.begin(), path.end());
print(path);

f(0,3) = -8
   0    6    7    8    5    3 


Preverimo še zaznavanje negativnega cikla na enakem primeru kot pri Bellman-Fordu, kjer spremenimo predznak povezavi iz vozlišča 7 v 1. Iz izpisa na diagonali vidimo, da so lahko del negativnih ciklov vsa vozlišča z izjemo 0 in 6. Vendar je tudi vozlišče 6 lahko del negativnega cikla 6-(7-1-2-4)*-7-1-6, le da algoritem ni ugotovil, da bi se to lahko zgodilo v primeru več krogov po manjšem negativnem ciklu 7-1-2-4. Omeniti velja tudi negativno dolžino izračunane poti iz vozlišča 8 nazaj do sebe, ki ima presenetljivo veliko absolutno vrednost.

In [36]:
mat[7][1]*=-1;

In [37]:
vector<VI> dist, prev;
FloydWarshall(mat,dist,prev);
print2(dist);

Negative cycle detected.
   0  -11  -10  -13   -8    7   -7  -17  -68 
 INF  -10   -9  -12   -7    8    1  -16  -67 
 INF  -11  -10  -13   -8    7    0  -17  -68 
 INF   -3   -2   -5    0   15    8   -9  -60 
 INF  -13  -12  -15  -10    5   -2  -19  -70 
 INF  -23  -22  -25  -20   -5  -12  -29  -80 
 INF   -4   -3   -6   -1   14    0  -10  -61 
 INF  -26  -25  -28  -23   -8  -15  -32  -83 
 INF  -71  -70  -73  -68  -53  -60  -77 -128 


## Primeri sorodnih problemov

- Tranzitivna ovojnica grafa (*transitive closure*)
- Štetje najkrajših poti (Dijkstra, Bellman-Ford, Floyd-Warshall)
- Arbitraža pri menjavi valut (*currency arbitrage*)
- Sistemi neenačb razlik (*difference constraints*)